## FASTAI + Swin Transformer + Stratified 10-Fold + Image Feature Normalization

__Introduction__

In this notebook we explore the using Swin Transformer with FastAI doing all the pre-processing tasks. This notebook also uses the timm library that helps with model creation and setting the weights for the pre-trained Swin model. Based on the analysis in the EDA notebook, we used stratified Kfold with bins instead of simple K-fold. 

__Kaggle_Score__ : 18.00088

In [1]:
# Required only if you are running the notebook on anaconda for the first time
#pip install timm

In [2]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
import gc
from sklearn.model_selection import StratifiedKFold
from fastai.vision.all import *


In [3]:
# To ensure results are reproducible
seed=42
set_seed(42, reproducible=True)
BATCH_SIZE = 32
N_FOLDS = 10
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True
dataset_path = Path('../input/petfinder-pawpularity-score/')

### Add file path to DF and normalize the output

In [4]:
def primary_preprocessing(data_type='train'):
    df = pd.read_csv(dataset_path/f'{data_type}.csv')
    df['path'] = df['Id'].map(lambda x:str(dataset_path/f'{data_type}'/x)+'.jpg')
    df = df.drop(columns=['Id'])
    if data_type=='train':
        df = df.sample(frac=1).reset_index(drop=True)
    return df
        

In [5]:
train_df = primary_preprocessing()
train_df['paw_norm'] = train_df['Pawpularity']/100
train_df.head()

In [6]:
test_df = primary_preprocessing(data_type='test')
test_df.head()

In [7]:
# Caching the model
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'


### Divide the data into bins based on Sturges' rule

In [8]:
num_bins = int(np.floor(1+np.log2(len(train_df))))
print(f"No. of bins: {num_bins}")
train_df['bins'] = pd.cut(train_df['paw_norm'], bins=num_bins, labels=False)

### Do a StratifiedKFold not based on Pawpopularity value but based on bins

In [9]:
train_df['fold'] = -1
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
train_df['fold'] = train_df['fold'].astype('int')

In [10]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

### Resize Image to 224,224 and nomarlize all characteristics of image i.e.  Brightness,Contrast Hue and Saturation

In [11]:
def get_data(fold):
    train_df_f = train_df.copy()
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(train_df_f, 
                                   valid_col = 'is_valid', 
                                   seed = seed, 
                                   fn_col = 'path', #filename/path is in the second column of the DataFrame
                                   label_col = 'paw_norm', #label is in the first column of the DataFrame
                                   y_block = RegressionBlock, #The type of target
                                   bs = BATCH_SIZE, #pass in batch size
                                   num_workers = 8,
                                   item_tfms = Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls


In [12]:
def get_learner(fold_num):
    data = get_data(fold_num)
    model = create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=data.c)
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    return learn

In [13]:
all_preds = []

for i in range(N_FOLDS):
    print(f'Fold {i} results')
    learn = get_learner(fold_num=i)
    learn.fit_one_cycle(5, 2e-5, cbs=[SaveModelCallback(), EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=2)]) 
    learn.recorder.plot_loss()
    
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=42, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='paw_norm', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    
    test_dl = dls.test_dl(test_df)
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    all_preds.append(preds)
    del learn
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
preds = np.mean(np.stack(all_preds), axis=0)
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)

__Final Observations__
1. The model results in the highest kaggle score from the nothebooks we ran
2. Use of stratified K-fold validation improves the accuracy of the model the most.
3. FastAI provides easy access to functions that can do a lot of pre-processing with very little steps.